In [ ]:
import numpy as np
from astropy.table import Table
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from exod.utils.plotting import cmap_image

In [ ]:
from exod.processing.detector import Detector, plot_var_with_regions
from exod.pre_processing.data_loader import DataLoader
from exod.utils.path import save_df
from exod.xmm.observation import Observation
from exod.processing.detector import Detector
from exod.utils.logger import logger
from photutils.detection import daofinder, irafstarfinder, starfinder

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils.aperture import CircularAperture
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord

from scipy.stats import binned_statistic_dd
from scipy.interpolate import interp1d
from astropy.table import unique

In [ ]:
tab_src = Table.read('../data/raw/0886121001/P0886121001EPX000OBSMLI0000.FTZ')
tab_src

In [ ]:
tab_src2 = Table.read('../data/raw/0886121001/P0886121001EPX000OMSRLI0000.FTZ')
tab_src2

In [ ]:
image = fits.open('../data/processed/0886121001/P0886121001PNS003PIEVLI0000_IMG.fits')[0].data
#image = fits.open('../data/processed/0886121001/P0886121001M1S001MIEVLI0000_IMG.fits')[0].data
#image = fits.open('../data/processed/0886121001/P0886121001M2S002MIEVLI0000_IMG.fits')[0].data

In [ ]:
# Create the Observation class
obsid = '0886121001'

obsid         = obsid
size_arcsec   = 20.0
time_interval = 250
gti_only      = True
gti_threshold = 0.5
min_energy    = 0.5
max_energy    = 12.0
sigma         = 4
clobber       = False

observation = Observation(obsid)
observation.get_files()

# Get the eventslist & image to use
event_list = observation.events_processed_pn[0]
event_list.read()

img = observation.images[0]
img.read(wcs_only=True)

# Initialize the Data Loader
dl = DataLoader(event_list=event_list,
                size_arcsec=size_arcsec,
                time_interval=time_interval,
                gti_only=gti_only,
                gti_threshold=gti_threshold,
                min_energy=min_energy,
                max_energy=max_energy)
dl.run()

# Create Data Cube
# dl.data_cube.plot_cube_statistics()
# dl.data_cube.video(savepath=None)


In [ ]:
h = fits.open('../data/raw/0886121001/P0886121001EPX000OMSRLI0000.FTZ')[0].header
h1 = fits.open('../data/raw/0886121001/P0886121001EPX000OMSRLI0000.FTZ')[1].header
XBINSIZE = h['XBINSIZE'] # Image Pixel Size (arcsec)
YBINSIZE = h['YBINSIZE'] # Image Pixel Size (arcsec)

In [ ]:
tab_src2_unique = unique(tab_src2, keys=['ML_ID_SRC'], keep='last')
tab_src2_unique

In [ ]:
img.wcs

In [ ]:
data_cube = dl.data_cube

# Only include bright sources
PN_CTS_MIN = 0
sub = tab_src[tab_src['PN_CTS'] > PN_CTS_MIN]

# Create skycoord
sc = SkyCoord(ra=sub['RA'], dec=sub['DEC'])

# Get Image Coordinates
x_img, y_img = img.wcs.world_to_pixel(sc)

# Convert to Sky Coordinates
X = x_img * 80
Y = y_img * 80

# Remove values outside the cube
xcube_max = dl.data_cube.bin_x[-1]
ycube_max = dl.data_cube.bin_y[-1]
XY = np.array([[x, y] for x, y in zip(X,Y) if (x < xcube_max) and (y < ycube_max)]).T
X = XY[0]
Y = XY[1]

# Interpolate to Cube coordinates & Round to int
interp_x_cube = interp1d(x=data_cube.bin_x, y=range(data_cube.shape[0]))
interp_y_cube = interp1d(x=data_cube.bin_y, y=range(data_cube.shape[1]))
x_cube = interp_x_cube(X)
y_cube = interp_y_cube(Y)
x_cube = np.round(x_cube).astype(int)
y_cube = np.round(y_cube).astype(int)

# Plot
im = np.nansum(data_cube.data, axis=2)
plt.figure(figsize=(10,10))
plt.imshow(im.T, origin='lower')
plt.scatter(x_cube, y_cube, color='red', marker='x', s=5)


for x, y in zip(x_cube, y_cube):
    im[x-1:x+1, y-1:y+1] = 0

    
plt.figure(figsize=(10,10))
plt.imshow(im.T, origin='lower')
#plt.scatter(x_cube, y_cube, color='red', marker='x', s=5)

In [ ]:
tab_positions = tab_src2['X_IMA','X_IMA_ERR','Y_IMA','Y_IMA_ERR']
tab_positions = unique(tab_positions)
tab_positions

In [ ]:
tab_raw  = tab_src2['RAWX', 'RAWY', 'CCDNR']
mask_nan = np.isnan(tab_src2['RAWX']).mask
tab_raw  = tab_raw[~mask_nan]
tab_raw = unique(tab_raw)
tab_raw

In [ ]:
X      = tab_src2['X_IMA']
Y      = tab_src2['Y_IMA']
X_ERR  = tab_src2['X_IMA_ERR'] 
Y_ERR  = tab_src2['Y_IMA_ERR']
RAWX   = tab_src2['RAWX']
RAWY   = tab_src2['RAWY']
CUTRAD = tab_src2['CUTRAD']

In [ ]:
print('Removing sources from event_list')
table1 = dl.event_list.data
table2 = tab_raw
pre = len(table1)
print(f'len pre: {pre}')
matching_columns = ['RAWX', 'RAWY', 'CCDNR']
mask = np.isin(table1[matching_columns], table2[matching_columns], invert=True)
filtered_table1 = table1[mask]
post = len(filtered_table1)
print(f'len post: {post} ({post-pre} rows)')
filtered_table1

In [ ]:
from astropy.coordinates import SkyCoord